In [1]:
#pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
directory = "TImages/"

In [4]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [5]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [6]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [7]:
create_training_data()

In [8]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [9]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [12]:
## SVM MODEL

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [14]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [32]:
# Hyperparameter Tuning using GridSearchCV
# param_grid = {'C': [0.1, 1, 10],
#               'kernel': ['linear', 'rbf', 'poly'],
#               'gamma': ['scale', 'auto', 0.1, 1, 10]
#               }
param_grid = {'C': [1,5,20],
              'kernel': ['rbf'],
              'gamma': ['auto']
              }

In [33]:
# Train the SVM model
svm = SVC()
grid_search = GridSearchCV(svm, param_grid, cv=5, verbose=2)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=24.3min
[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=22.2min
[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=20.8min
[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=18.3min
[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=20.2min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time=18.4min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time=20.4min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time=20.6min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time=19.6min
[CV] END ........................C=5, gamma=auto, kernel=rbf; total time=36.6min
[CV] END .......................C=20, gamma=auto, kernel=rbf; total time=23.5min
[CV] END .......................C=20, gamma=auto,

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 5, 20], 'gamma': ['auto'], 'kernel': ['rbf']},
             verbose=2)

In [34]:
# Get the best estimator and evaluate it
best_svm = grid_search.best_estimator_
y_test_pred = best_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best SVM Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Best SVM Parameters: {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}
Accuracy: 94.44%


In [ ]:
# work progressing...........


In [35]:
grid_search.cv_results_

{'mean_fit_time': array([527.60728788, 497.96975718, 535.66765103]),
 'std_fit_time': array([116.13762622,  95.52768956, 113.84415764]),
 'mean_score_time': array([ 741.14449115,  890.13505678, 1587.91948457]),
 'std_score_time': array([  45.04896093,  310.9618004 , 1686.81048976]),
 'param_C': masked_array(data=[1, 5, 20],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_gamma': masked_array(data=['auto', 'auto', 'auto'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'rbf', 'rbf'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'},
  {'C': 5, 'gamma': 'auto', 'kernel': 'rbf'},
  {'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}],
 'split0_test_score': array([0.93803419, 0.94978632, 0.9508547 ]),
 'split1_test_score': array([0.93696581, 0.94978632, 0.95

In [36]:
df = pd.DataFrame(grid_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,527.607288,116.137626,741.144491,45.048961,1,auto,rbf,"{'C': 1, 'gamma': 'auto', 'kernel': 'rbf'}",0.938034,0.936966,0.936966,0.940171,0.934829,0.937393,0.001736,3
1,497.969757,95.527690,890.135057,310.961800,5,auto,rbf,"{'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}",0.949786,0.949786,0.956197,0.958333,0.949786,0.952778,0.003726,1
2,535.667651,113.844158,1587.919485,1686.810490,20,auto,rbf,"{'C': 20, 'gamma': 'auto', 'kernel': 'rbf'}",0.950855,0.950855,0.955128,0.955128,0.951923,0.952778,0.001958,1


In [37]:
df[['param_C', 'param_kernel', 'mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.937393
1,5,rbf,0.952778
2,20,rbf,0.952778


In [38]:
grid_search.best_score_

0.9527777777777778

In [39]:
grid_search.best_params_

{'C': 5, 'gamma': 'auto', 'kernel': 'rbf'}